In [1]:
from dask.distributed import Client, LocalCluster
import os
import datetime as dt
import pandas as pd
import requests
from tsfresh import extract_features, select_features, extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.dataframe_functions import roll_time_series
import xgboost as xgb
import numpy as np
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import matplotlib
import dask.dataframe as dd
from dask.dataframe import from_pandas
from dask.distributed import progress
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import multiprocessing
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats


In [ ]:
cpu_cores = multiprocessing.cpu_count()
os.makedirs('dask-scraFONh', exist_ok=True)
cluster = LocalCluster(local_directory='dask-scraFONh', n_workers= cpu_cores)
client = Client(cluster)
# if torch.cuda.is_available():
#     current_device = torch.cuda.current_device()
#     device_name = torch.cuda.get_device_name(current_device)
#     print(f"CUDA is using: {device_name}")
# else:
#     print("CUDA is not available.")

2024-06-04 21:41:48,322 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 5 memory: 385 MB fds: N/A>>
Traceback (most recent call last):
  File "c:\Crypto_Price_Action\trading_venv\Lib\site-packages\tornado\ioloop.py", line 937, in _run
    val = self.callback()
          ^^^^^^^^^^^^^^^
  File "c:\Crypto_Price_Action\trading_venv\Lib\site-packages\distributed\system_monitor.py", line 210, in update
    gpu_metrics = nvml.real_time()
                  ^^^^^^^^^^^^^^^^
  File "c:\Crypto_Price_Action\trading_venv\Lib\site-packages\distributed\diagnostics\nvml.py", line 329, in real_time
    "utilization": _get_utilization(h),
                   ^^^^^^^^^^^^^^^^^^^
  File "c:\Crypto_Price_Action\trading_venv\Lib\site-packages\distributed\diagnostics\nvml.py", line 298, in _get_utilization
    return pynvml.nvmlDeviceGetUtilizationRates(h).gpu
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Crypto_Price_Action\tra

In [ ]:
def get_proj_ids(coins, project_url_df):
    list_id = []
    for coin in coins:
        list_id.append(project_url_df.loc[project_url_df['symbol']== coin,'project_id'].item())
    return list_id

def get_project_url_df():
    url = "https://api.tokenterminal.com/v2/projects"
    headers = {
    "accept": "application/json",
    "authorization": TOKENTERMINAL_API_KEY
    }
    response = requests.get(url, headers=headers)
    list_of_dicts= eval(response.text.replace('null', 'None'))
    return pd.DataFrame(list_of_dicts['data'])

def get_specific_urls(coins, project_url_X):
    spec_urls = project_url_df.loc[project_url_df['symbol'].isin(coins), 'url'].tolist()
    return spec_urls

def get_df_of_OCA(coins, date, project_url_df):
    list_dd = []
    headers = {
        "accept": "application/json",
        "authorization": TOKENTERMINAL_API_KEY
        }
    for url in get_specific_urls(coins, project_url_df):
        response = requests.get(f'https://api.tokenterminal.com{url}/metrics', headers=headers)
        data = response.text.replace("null", "None")
        raw_data_string = data[response.text.find("['")+9:response.text.find("']")]
        try: raw_data = eval(raw_data_string)
        except: continue
        list_dd.append(pd.DataFrame(raw_data))
    FON = list_dd[0]
    count = 0
    for list in list_dd:
        if count == 0: 
            count = count +1
            continue
        else:
            FON= pd.merge(left = FON, right= list_dd[count], how = 'outer', suffixes = (None, coins[count]))
            count = count + 1
            continue
    # FON.drop(['project_name'], axis=1, inplace = True)
    FON['timestamp']=FON['timestamp'].apply(pd.to_datetime)
    FON= FON[(FON['timestamp'] >= date)]
    npartitions = FON['project_name'].nunique()
    return dd.from_pandas(FON.set_index('project_name'), npartitions=npartitions)

def roll(FON, timeshift, coins):
    FON.dropna(subset = 'forecast', inplace=True)
    FON.dropna(subset = 'price', inplace=True)
    FON[FON.columns.drop(['timestamp', 'project_id','forecast'])] = FON.drop(['timestamp', 'project_id','forecast'], axis =1).ffill()
    X = FON 
    FON_rolled = roll_time_series(X, column_id='project_id', column_sort='timestamp', max_timeshift=timeshift,  n_jobs =1)
    # for column in FON_rolled.columns:
    #     if FON_rolled[column].dtypes =='float64' or  FON_rolled[column].dtypes == 'int64': 
    #         FON_rolled[column] = FON_rolled[column].fillna(FON_rolled[column].mean())
    # FON_rolled.set_index(by = 'project_id', inplace = True)
    # return dd.from_pandas(FON_rolled)
    return FON_rolled.drop('project_id', axis=1)

def train_xgbmodels_on_features(SF, days_forecast):
    for key in SF.keys():
        df = SF[key].dropna()
        x= df.drop('forecast', axis =1)
        y = df['forecast']
        Y_train= y.iloc[ :int(0.85*len(df))]
        Y_test = y.iloc[int(0.85*len(df))+1: ]
        X_train = x.iloc[ : int(0.85*len(df))]
        X_test = x.iloc[int(0.85*len(df))+1 : ]
        xgbmodel = xgb.XGBRegressor(device = 'cuda', max_depth = 1000000000).fit(X_train, Y_train, eval_set = [(X_train, Y_train), (X_test, Y_test)], verbose=False)
        Y_pred= xgbmodel.predict(X_test)
        # score= np.sqrt(mean_squared_error(Y_test, Y_pred)) / Y_pred
        score = root_mean_squared_error(Y_test, Y_pred)
        sm = (score)/(Y_test.mean())
        sstd= (score)/(Y_test.std())
        print(key + f' RMSE: {score} \n (score/mean):{sm} \n (score/std):{sstd}')
        pd.concat([pd.DataFrame(Y_pred, index= Y_test.index, columns= ['prediction']), Y_test.rename("realized")], axis=1).plot(title= f"{key}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days' )

def calc_log_returns_and_vol(FON, days_forecast, target):
    FON.sort_values(by= 'timestamp', inplace=True)
    FON['LOG_R'] = np.log(FON['price'])- np.log(FON['price'].shift(days_forecast))
    FON['EWM_VOL_R'] = FON['LOG_R'].ewm(alpha=0.01).std()
    # FON['Rolling_VOL_R'] = FON['LOG_R'].rolling(window=days_forecast).std()
    FON['price_volatility'] = FON['price'].rolling(window=days_forecast).std() 
    FON['returns_volatility'] = FON['LOG_R'].rolling(window=days_forecast).std() 
    if target == 'price_vol':
        FON['forecast'] = FON['price_volatility'].shift(-days_forecast)
    elif target == 'log_returns':
        FON['forecast'] =  np.log(FON['price'].shift(-days_forecast))- np.log(FON['price'])
    elif target == 'price_absolute_change':
        FON['forecast'] = abs(FON['price'].shift(-days_forecast)- FON['price'])
    elif target == 'price_shifted': 
        FON['forecast'] = FON['price'].shift(-days_forecast)
    elif target == 'LR_shifted':
        FON['forecast'] = FON['LOG_R'].shift(-days_forecast)
    elif target == 'ret_vol':
         FON['forecast'] = FON['returns_volatility'].shift(-days_forecast)
    elif target == 'ret_vol_diff':
        FON['forecast'] = FON['returns_volatility'].shift(-days_forecast).diff(days_forecast)
    elif target == 'VOL_LRabs':
        FON['forecast'] = abs(FON['LOG_R']).shift(-days_forecast)
    elif target == 'VOL_LRdiff':
        FON['forecast'] = abs(FON['LOG_R']).rolling(days_forecast).std().shift(-days_forecast)
    elif target == 'pct_change':
        FON['forecast'] = FON['price'].pct_change(days_forecast)
    else:
        print('Invalid target')
    return FON

def add_inf_int(X, Sup_features):
    X = X.merge(Sup_features, how = 'left', on = 'timestamp', suffixes = (None, None))
    X[X.columns.drop]
    # X.replace('<NA>', np.NaN, regex=True)
    return X
def merge_BTC_feat(part, BTC_feat):
    if part['project_id'].unique() != 'bitcoin':
        X = part.merge(BTC_feat, how = 'left', on = 'timestamp', suffixes=(None, '_BTC'))
    else: 
        X = part
    return X

# def add_Sup_features(FON):
#     # Sup_features = pd.read_csv("C:\Crypto_Price_Action\SUPP_FEATURES\All_supplementary_features.csv")
#     # Sup_features['TIME_PERIOD'] = Sup_features['TIME_PERIOD'].apply(pd.to_datetime)
#     # Sup_features = Sup_features[['REF_AREA', 'MEASURE', 'OBS_VALUE', 'TIME_PERIOD']]
#     # for measure in Sup_features['MEASURE'].unique():
#     #     for REFAREA in Sup_features[Sup_features['MEASURE']== measure]['REF_AREA'].unique():
#     #         X = Sup_features[Sup_features['MEASURE']== measure][['REF_AREA', 'OBS_VALUE','TIME_PERIOD']].groupby(['REF_AREA', 'TIME_PERIOD']).mean()
#     #         X=  X.loc[REFAREA]
#     #         X.rename(columns = {'OBS_VALUE': f'last_month: {measure}_{REFAREA}'}, inplace = True)
#     #         start_date = X.index.min() 
#     #         end_date = X.index.max() + pd.DateOffset(day=62)
#     #         dates = pd.date_range(start_date, end_date, freq='D')
#     #         dates.name = 'timestamp'
#     #         X = X.reindex(dates, method='ffill').tz_localize('UTC')
#     #         X.index = X.index + pd.Timedelta(days=32)
    
#             FON = FON.merge(X, how = 'left', on ='timestamp')
#     return FON

In [4]:
L1= ['BTC','ETH']
    #   ,'BNB','MATIC','SOL',TRX, 'ADA', 'XRP']
projects= ['BTC', 'CAKE', 'JUP', 'UNI', 'AAVE']
L2 = ['BTC' 'ARB', 'OP']
coins = L1  
days_forecast= 32
# coins = ['SOL','BNB','ETH', 'ADA', 'BTC']
date = pd.to_datetime(dt.datetime.today() -pd.Timedelta(days=days_forecast), utc =True) - pd.Timedelta(150, 'W')
timeshift = 500
project_url_df = get_project_url_df()  
project_ids = get_proj_ids(coins, project_url_df)
names = sorted(project_ids)


In [5]:
Sup_features = pd.read_csv(r"C:\Crypto_Price_Action\5YF_inflation_expectations.csv").merge(pd.read_csv(r"C:\Crypto_Price_Action\5Y_FFER.csv"), on = 'DATE')
Sup_features['DATE'] = pd.to_datetime(Sup_features['DATE'], utc=True)
Sup_features.rename({'DATE':'timestamp','T5YIE':'5Y_inflation_expectations_US', 'DFF':'Federal_Funds_Rate'}, inplace=True, axis=1)
Sup_features[Sup_features.drop('timestamp', axis=1).columns] = Sup_features[Sup_features.drop('timestamp', axis=1).columns].apply(pd.to_numeric, errors='coerce')
# Sup_features.set_index('timestamp', inplace = True)

In [6]:
target = 'price_vol'
FON = get_df_of_OCA(coins, date, project_url_df)
FON= FON.map_partitions(calc_log_returns_and_vol, days_forecast, target, enforce_metadata=False).map_partitions(add_inf_int, Sup_features, align_dataframes=False).persist()
BTC_feat = FON.partitions[names.index('bitcoin')][['LOG_R', 'EWM_VOL_R', 'price_vol', 'ret_vol', 'timestamp']].set_index('timestamp')
FON = FON.map_partitions(merge_BTC_feat, BTC_feat, align_dataframes=False).persist()

KeyError: "['price_std', 'Annualized_volatility'] not in index"

In [ ]:
relevance= abs(FON.partitions[1].drop(['timestamp', 'project_id'], axis =1).compute().ffill().dropna(how='all').dropna(subset='forecast').corr()['forecast'])
relevance.sort_values(ascending=False, inplace=True)
relevance

forecast                        1.000000
Annualized_volatility           0.544812
Annualized_volatility_BTC       0.410728
LOG_R                           0.270943
user_wau                        0.199137
LOG_R_BTC                       0.146268
user_dau                        0.143627
EWM_VOL_R                       0.128133
transaction_count               0.123589
transactions_per_second         0.123589
earnings                        0.103707
token_trading_volume            0.102770
EWM_VOL_R_BTC                   0.099906
user_mau                        0.097710
price_std                       0.095704
tokenholders                    0.091376
active_developers               0.088832
expenses                        0.086479
token_incentives                0.086479
block_time                      0.069898
fees_supply_side                0.056863
pf_fully_diluted                0.056503
pf_circulating                  0.056503
ps_fully_diluted                0.053847
ps_circulating  

In [8]:
list_rev_var = []
for (key,value) in relevance.items(): 
    if value >= 0.10: list_rev_var.append(key)
# list_rev_var.remove('LOG_R_BTC')
# list_rev_var.remove('price_std')
# list_rev_var.remove('market_cap_circulating')
# list_rev_var.remove('market_cap_fully_diluted')
# list_rev_var.remove('market_cap_circulating')
# list_rev_var.remove('pf_fully_diluted')
# list_rev_var.remove('token_turnover_fully_diluted')
# list_rev_var.remove('token_turnover_circulating')
# list_rev_var.remove('ps_circulating')
# list_rev_var.remove('ps_fully_diluted')
# list_rev_var.remove('pf_circulating')
# list_rev_var.remove('transaction_fee_average')
# list_rev_var.remove('token_incentives')
# list_rev_var.remove('fees')
list_rev_var

['forecast',
 'Annualized_volatility',
 'Annualized_volatility_BTC',
 'LOG_R',
 'user_wau',
 'LOG_R_BTC',
 'user_dau',
 'EWM_VOL_R',
 'transaction_count',
 'transactions_per_second',
 'earnings',
 'token_trading_volume']

In [9]:
relevance_first_diff= abs(FON.partitions[1].drop(['timestamp', 'project_id'], axis =1).compute().ffill().diff().dropna(how='all').dropna(subset='forecast').corr()['forecast'])
relevance_first_diff.sort_values(ascending=False, inplace=True)
list_rev_var_FD = []
for (key,value) in relevance_first_diff.items(): 
    if value >= 0.10: list_rev_var_FD.append(key)
list_rev_var_FD
# list_rev_var.remove('market_cap_circulating')
# list_rev_var.remove('market_cap_fully_diluted')
# list_rev_var.remove('market_cap_circulating')
# list_rev_var.remove('pf_fully_diluted')
# list_rev_var.remove('token_turnover_fully_diluted')
# list_rev_var.remove('token_turnover_circulating')
# list_rev_var.remove('ps_circulating')
# list_rev_var.remove('ps_fully_diluted')
# list_rev_var.remove('pf_circulating')
# list_rev_var.remove('transaction_fee_average')
# list_rev_var.remove('token_incentives')
# list_rev_var.remove('fees')
list_rev_var_FD

['forecast', 'Annualized_volatility', 'Annualized_volatility_BTC']

In [10]:
# variables = list_rev_var 
variables = list_rev_var_FD + list_rev_var
variables = pd.Series(variables).unique().tolist()
variables.append('timestamp')
variables.append('id')
rolled = FON.map_partitions(roll, timeshift, coins, enforce_metadata=False).persist()
ETH = rolled.partitions[1].compute()[variables]

Rolling: 100%|██████████| 1/1 [00:00<00:00, 28.82it/s]


In [11]:
ETH

,forecast,Annualized_volatility,Annualized_volatility_BTC,LOG_R,user_wau,LOG_R_BTC,user_dau,EWM_VOL_R,transaction_count,transactions_per_second,earnings,token_trading_volume,timestamp,id
0,0.015567,3.528158,2.819630,0.594687,1790530.0,0.495174,366972.0,0.184221,1186758.0,13.735625,-3.725621e+07,2.207905e+10,2021-08-21 00:00:00+00:00,"('ethereum', Timestamp('2021-08-21 00:00:00+00..."
1,0.015567,3.528158,2.819630,0.594687,1790530.0,0.495174,366972.0,0.184221,1186758.0,13.735625,-3.725621e+07,2.207905e+10,2021-08-21 00:00:00+00:00,"('ethereum', Timestamp('2021-08-22 00:00:00+00..."
2,0.172479,3.336009,2.649995,0.492029,1805806.0,0.446043,349145.0,0.184230,1183893.0,13.702465,-3.791127e+07,1.933919e+10,2021-08-22 00:00:00+00:00,"('ethereum', Timestamp('2021-08-22 00:00:00+00..."
3,0.015567,3.528158,2.819630,0.594687,1790530.0,0.495174,366972.0,0.184221,1186758.0,13.735625,-3.725621e+07,2.207905e+10,2021-08-21 00:00:00+00:00,"('ethereum', Timestamp('2021-08-23 00:00:00+00..."
4,0.172479,3.336009,2.649995,0.492029,1805806.0,0.446043,349145.0,0.184230,1183893.0,13.702465,-3.791127e+07,1.933919e+10,2021-08-22 00:00:00+00:00,"('ethereum', Timestamp('2021-08-23 00:00:00+00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368731,0.965802,1.891899,2.059888,-0.069931,2135337.0,-0.090984,412134.0,0.180094,1140939.0,13.205312,-6.105253e+06,1.273985e+10,2024-04-28 00:00:00+00:00,"('ethereum', Timestamp('2024-05-02 00:00:00+00..."
368732,1.159230,1.730639,1.843219,-0.103393,2197790.0,-0.116373,465874.0,0.180113,1224677.0,14.174502,-5.254995e+06,1.078619e+10,2024-04-29 00:00:00+00:00,"('ethereum', Timestamp('2024-05-02 00:00:00+00..."
368733,1.332302,1.612914,1.622215,-0.154422,2261304.0,-0.129247,474065.0,0.180693,1262466.0,14.611875,-3.151434e+06,1.548080e+10,2024-04-30 00:00:00+00:00,"('ethereum', Timestamp('2024-05-02 00:00:00+00..."
368734,1.468212,1.507453,1.544615,-0.165006,2320061.0,-0.186818,442784.0,0.181373,1206626.0,13.965579,-4.702621e+06,1.925116e+10,2024-05-01 00:00:00+00:00,"('ethereum', Timestamp('2024-05-02 00:00:00+00..."


In [12]:
# BTC_VOL = rolled.partitions[names.index('bitcoin')][['Annualized_volatility_30_day', 'timestamp']].groupby('timestamp').last()['Annualized_volatility_30_day'].compute()
# BTC_VOL.rename('BTC_EWM_VOL',inplace =True)
# BTC_RET = rolled.partitions[names.index('bitcoin')][['LOG_R', 'timestamp']].groupby('timestamp').last()['LOG_R'].compute()
# BTC_RET.rename('BTC_LOG_R',inplace =True)
# BTC_PRICE = rolled.partitions[names.index('bitcoin')][['price', 'timestamp']].groupby('timestamp').last()['price'].compute()
# BTC_PRICE.rename('BTC_PRICE',inplace =True)


In [13]:
df = ETH.dropna()
y= df.groupby('timestamp')[['forecast', 'timestamp', 'id']].first().set_index('id')  
X = df.drop(['forecast'], axis=1)
X= X.merge(y['forecast'], how='left', on=['id'])
X = X.dropna(subset=['forecast'], axis =0)
X[X.drop(['id','timestamp','forecast'], axis =1).columns] =X[X.drop(['id','timestamp','forecast'], axis =1).columns].fillna(method='ffill', axis=0)
X = X.dropna()
# default_fc_parameters=EfficientFCParameters()
FC = extract_relevant_features(X.drop('forecast', axis =1), X.groupby('id')['forecast'].last(), column_id='id', column_sort='timestamp', n_jobs= cpu_cores, hypotheses_independent=False, ml_task='regression', fdr_level=0.001, default_fc_parameters=EfficientFCParameters())
ETH = FC.merge(y[['forecast','timestamp']], how='left', left_index=True, right_index=True).set_index('timestamp')

c:\Crypto_Price_Action\trading_venv\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  if await self.run_code(code, result, async_=asy):
Feature Extraction: 100%|██████████| 100/100 [02:10<00:00,  1.30s/it]


In [14]:
ETH

,"Annualized_volatility__fft_aggregated__aggtype_""skew""","Annualized_volatility__fft_aggregated__aggtype_""kurtosis""","Annualized_volatility__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0","Annualized_volatility_BTC__fft_coefficient__attr_""real""__coeff_9","Annualized_volatility__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""min""","Annualized_volatility__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""min""","Annualized_volatility_BTC__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","Annualized_volatility__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""mean""","Annualized_volatility_BTC__fft_aggregated__aggtype_""skew""","Annualized_volatility__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""mean""",...,"Annualized_volatility__fft_coefficient__attr_""imag""__coeff_10","Annualized_volatility_BTC__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""mean""","Annualized_volatility_BTC__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""min""",Annualized_volatility_BTC__quantile__q_0.9,"Annualized_volatility_BTC__fft_coefficient__attr_""imag""__coeff_57","Annualized_volatility_BTC__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.8","Annualized_volatility__fft_coefficient__attr_""imag""__coeff_7",Annualized_volatility__index_mass_quantile__q_0.8,"Annualized_volatility_BTC__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""var""",forecast
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-08-04 00:00:00+00:00,2.573036,10.090206,0.000000,2.067922,-0.028571,-0.001884,0.000000,-0.293693,2.728376,-0.038861,...,-3.748275,0.033710,0.003716,2.393490,-0.361576,0.000000,-2.293445,1.000000,-0.100783,-1.819505
2021-08-05 00:00:00+00:00,2.573036,10.090206,0.000000,2.067922,-0.028571,-0.001884,0.000000,-0.293693,2.728376,-0.038861,...,-3.748275,0.033710,0.003716,2.366925,-0.361576,0.000000,-2.293445,1.000000,-0.100783,-1.450226
2021-08-06 00:00:00+00:00,2.573036,10.090206,-0.379343,2.067922,-0.028571,-0.001884,-0.265649,-0.293693,2.728376,-0.038861,...,-3.748275,0.033710,0.003716,2.340360,-0.361576,0.000000,-2.293445,1.000000,-0.100783,-0.953055
2021-08-07 00:00:00+00:00,2.573036,10.090206,-0.103183,2.067922,-0.028571,-0.001884,-0.265649,-0.293693,2.728376,-0.038861,...,-3.748275,0.033710,0.003716,2.313795,-0.361576,0.000000,-2.293445,1.000000,-0.100783,-0.686311
2021-08-08 00:00:00+00:00,2.573036,10.090206,-0.187300,2.067922,-0.028571,-0.001884,-0.288283,-0.293693,2.728376,-0.038861,...,-3.748275,0.033710,0.003716,2.287231,-0.361576,0.000000,-2.293445,0.800000,-0.100783,-0.367847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-15 00:00:00+00:00,2.729616,11.152956,-0.000762,-46.405374,0.359872,0.019452,-0.000037,0.031776,2.995865,0.002371,...,14.547876,0.031536,0.008059,1.720780,0.594890,0.026729,-22.325305,0.834331,-0.248734,1.257212
2024-05-16 00:00:00+00:00,2.734483,11.220023,-0.000998,-41.004322,0.284756,0.016437,-0.000022,0.035251,2.873110,0.002615,...,16.012529,0.031816,0.023333,1.720780,1.210023,0.026729,-25.532455,0.834331,-0.268527,1.030684
2024-05-17 00:00:00+00:00,2.723936,11.136693,-0.000930,-34.880507,0.278260,0.017304,0.000791,0.062729,2.781682,0.004619,...,17.250516,0.033147,0.033415,1.720780,1.700757,0.026729,-28.525533,0.832335,-0.245585,0.790420


In [15]:
# SF = {}
# count = 0
# for name in names:
#     if name != 'bitcoin':
#         count = names.index(name)
#         df = rolled.partitions[count].compute()
#         df.dropna(subset=['price'], inplace=True)
#         df.dropna(how='all', axis=0, inplace=True)
#         df.dropna(how='all', axis=1, inplace=True)
#         y= df.groupby('timestamp')[['forecast', 'timestamp', 'id']].first().set_index('id')  
#         X = df.drop(['forecast'], axis=1)
#         X = X.merge(BTC_VOL, how='left', on='timestamp').merge(BTC_RET, how='left', on='timestamp').merge(BTC_PRICE, how='left', on='timestamp')
#         X= X.merge(y['forecast'], how='left', on=['id'])
#         X = X.dropna(subset=['forecast'], axis =0)
#         X[X.drop(['id','timestamp','forecast'], axis =1).columns] =X[X.drop(['id','timestamp','forecast'], axis =1).columns].fillna(method='ffill', axis=0)
#         X = X.dropna()
#         FC = extract_relevant_features(X.drop('forecast', axis =1), X.groupby('id')['forecast'].last(), column_id='id', column_sort='timestamp', n_jobs= cpu_cores, hypotheses_independent=False, ml_task='regression', fdr_level=0.01, default_fc_parameters=EfficientFCParameters())
#         SF[name] = FC.merge(y[['forecast','timestamp']], how='left', left_index=True, right_index=True).set_index('timestamp')
#     else:
#         continue

In [14]:
ETH_rel_table= calculate_relevance_table(ETH.drop('forecast', axis =1), ETH['forecast'], n_jobs= cpu_cores).sort_values(by='p_value', ascending= True)

In [15]:
ETH_rel_table.head(500)

,feature,type,p_value,relevant
feature,,,,
"Annualized_volatility__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0",Annualized_volatility__change_quantiles__f_agg...,real,8.256606e-56,True
Annualized_volatility__mean_change,Annualized_volatility__mean_change,real,8.422729e-52,True
"Annualized_volatility__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0",Annualized_volatility__change_quantiles__f_agg...,real,9.461543e-52,True
Annualized_volatility_BTC__energy_ratio_by_chunks__num_segments_10__segment_focus_9,Annualized_volatility_BTC__energy_ratio_by_chu...,real,1.321777e-50,True
"Annualized_volatility__fft_aggregated__aggtype_""kurtosis""",Annualized_volatility__fft_aggregated__aggtype...,real,1.107028e-45,True
...,...,...,...,...
"Annualized_volatility__fft_coefficient__attr_""abs""__coeff_96","Annualized_volatility__fft_coefficient__attr_""...",real,3.485671e-10,True
"Annualized_volatility__fft_coefficient__attr_""angle""__coeff_86","Annualized_volatility__fft_coefficient__attr_""...",real,3.555378e-10,True
"EWM_VOL_R__fft_coefficient__attr_""imag""__coeff_73","EWM_VOL_R__fft_coefficient__attr_""imag""__coeff_73",real,3.555912e-10,True


In [18]:
# relevance_tables = {}
# for name in names:
#     count= 0 
#     if name != 'bitcoin':
#         X = SF[name].dropna(subset=['forecast'])
#         relevance_tables[name]= calculate_relevance_table(X.drop('forecast', axis =1), X['forecast'], n_jobs= cpu_cores).sort_values(by='p_value', ascending=True)
#         count = count + 1
#     else:
#         count = count + 1
#         continue

In [19]:
topSF = ETH[ETH_rel_table.head(500)['feature'].values].merge(ETH['forecast'], how='left', on='timestamp')
# topSF = SF['ethereum'][relevance_tables['ethereum'].head(1000)['feature'].values].merge(SF['ethereum']['forecast'], how='left', on='timestamp')

In [18]:
# merge drift
# variables.remove('forecast')
# variables.remove('timestamp')
# variables.remove('id')
ETH = ETH.merge(abs(FON.partitions[1].compute().set_index('timestamp')['returns_volatility']).rename('returns_volatility'), on = 'timestamp', how='left', suffixes=(None, None))
ETH.columns.nunique() - len(ETH.columns)

KeyError: 'returns_volatility'

In [25]:
ETH[['forecast', 'Annualized_volatility']].tail(20)

,forecast,Annualized_volatility
timestamp,,
2024-04-30 00:00:00+00:00,-0.288262,1.161416
2024-05-01 00:00:00+00:00,-0.229273,1.118172
2024-05-02 00:00:00+00:00,-0.200246,1.141149
2024-05-03 00:00:00+00:00,-0.243907,1.175329
2024-05-04 00:00:00+00:00,-0.313353,1.200613
2024-05-05 00:00:00+00:00,0.098114,1.210617
2024-05-06 00:00:00+00:00,0.578414,1.201720
2024-05-07 00:00:00+00:00,0.910842,1.193587
2024-05-08 00:00:00+00:00,1.266080,1.091817


In [16]:
xgb_model = xgb.XGBRegressor(device = 'cuda',  verbosity=3, n_jobs= -1,  eval_metric='mae',  n_estimators=7500, early_stopping_rounds=25)
param_dist = {
    'max_depth': [3, 5, 10, 25, 50, 100, 200],
    # 'n_estimators': range(5, 205, 25),
    # 'subsample': stats.uniform(0.5, 0.5),
    'gamma': np.arange(0, 2, 0.001).tolist(),
    # 'num_parallel_tree': [1,3,5,10],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 0.7, 1],
    'colsample_by_tree': np.arange(0.1, 1.0, 0.1).tolist(),
    'feature_selector': ['cyclic', 'greedy', 'thrifty'],
    # 'top_k': range(5, 205, 55),
}
splits= 3
random_search = RandomizedSearchCV(
                                    xgb_model, param_distributions=param_dist, n_iter= 15, refit = 'neg_mean_absolute_error', cv=splits,
                                    verbose=3,
                                    scoring=['r2','neg_mean_absolute_error'], 
                                    n_jobs= 1
                                   )
# scoring params:
# 'explained_variance'
# 'max_error'
# 'neg_mean_absolute_error'
# 'neg_mean_squared_error'
# 'neg_root_mean_squared_error'
# 'neg_mean_squared_log_error'
# 'neg_root_mean_squared_log_error'
# 'neg_median_absolute_error'
# 'r2'
# 'neg_mean_poisson_deviance
# 'neg_mean_gamma_deviance'
# 'neg_mean_absolute_percentage_error'
# 'd2_absolute_error_score'
# 'd2_pinball_score'
# 'd2_tweedie_score'

In [17]:
name='ethereum'
X= ETH
# X = SF[name].dropna(subset=['forecast']).set_index('timestamp')
X_train, X_test, Y_train, Y_test = train_test_split(X.drop('forecast', axis=1), X['forecast'], test_size= 0.2, shuffle=False)
base_model = xgb_model.fit(X_train, Y_train, eval_set = [(X_test, Y_test)], verbose=3)
Y_predbs=pd.Series(base_model.predict(X_test), X_test.index).rename('prediction')
pd.concat([Y_predbs, Y_test.rename("realized"), X_test['price_vol'].rename("current")], axis=1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days')
# pd.concat([Y_predbs, Y_test.rename("realized")], axis =1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days')
base_model.score(X_test, Y_test)

[21:16:44] ======== Monitor (0): HostSketchContainer ========
[21:16:44] AllReduce: 0.008748s, 1 calls @ 8748us

[21:16:44] MakeCuts: 0.015066s, 1 calls @ 15066us

[21:16:45] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[21:16:45] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\tree\updater_gpu_hist.cu:744: [GPU Hist]: Configure
[21:16:45] ======== Monitor (0):  ========
[21:16:45] InitCompressedData: 0.001098s, 1 calls @ 1098us

[21:16:45] ======== Monitor (0):  ========
[21:16:45] InitCompressedData: 0.012588s, 1 calls @ 12588us

[0]	validation_0-mae:0.78910
[3]	validation_0-mae:0.78364
[6]	validation_0-mae:0.81166
[9]	validation_0-mae:0.83613
[12]	validation_0-mae:0.84587
[15]	validation_0-mae:0.85395
[18]	validation_0-mae:0.87449
[21]	validation_0-mae:0.87872
[24]	validation_0-mae:0.879

c:\Crypto_Price_Action\trading_venv\Lib\site-packages\xgboost\sklearn.py:1168: UserWarning: [21:16:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  predts = self.get_booster().inplace_predict(


KeyError: 'Annualized_volatility'

In [ ]:
Model_A = xgb.XGBRegressor(device = 'cuda',  verbosity=3, n_jobs= -1, objective='reg:squaredlogerror', eval_metric='rmsle',  n_estimators=7500, early_stopping_rounds=100, max_depth=5000, learning_rate=0.7, gamma=0.0012)
Model_A.fit(X_train, Y_train, eval_set = [(X_test, Y_test)], verbose=3)
Y_PredA =pd.Series(Model_A.predict(X_test), X_test.index).rename('prediction')
pd.concat([Y_PredA, Y_test.rename("realized"), X_test['Annualized_volatility'].rename("current")], axis=1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days')
# pd.concat([Y_predbs, Y_test.rename("realized")], axis =1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days')
Model_A.score(X_test, Y_test)

In [ ]:
opti_xgbmodel = random_search.fit(X_train, Y_train, eval_set = [(X_test, Y_test)], verbose=3)
print(f"{name} Best set of hyperparameters: ", opti_xgbmodel.best_params_)
print(f"{name} Best score: ", opti_xgbmodel.best_score_)
Y_pred= pd.Series(opti_xgbmodel.predict(X_test), X_test.index).rename('prediction')


In [ ]:
pd.concat([Y_pred, Y_test.rename("realized"), X_test['VOL'].rename("current")], axis=1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days')


In [ ]:
pd.DataFrame(opti_xgbmodel.cv_results_).set_index('rank_test_r2').sort_index(ascending=True).iloc[0].params

In [ ]:
pd.DataFrame(opti_xgbmodel.cv_results_).set_index('rank_test_r2').sort_index(ascending=True).iloc[0].params

In [ ]:
xgb_model.score(X_test, Y_test)

In [ ]:
xgb_model.fit(X_train, Y_train, eval_set = [(X_test, Y_test)], verbose=3).predict(X_test)

In [ ]:
xgb_model = xgb.XGBRegressor(device = 'cuda',  verbosity=3, n_jobs= -1, objective='reg:squaredlogerror', eval_metric='rmsle',  n_estimators=7500, early_stopping_rounds=25,num_parallel_tree=5, max_depth=100, learning_rate=0.005, gamma=0.08, colsample_by_tree=0.7000000000000001)

In [ ]:
params = results.iloc[0].params
params

In [ ]:
# prev 
results = pd.DataFrame(opti_xgbmodel.cv_results_).set_index('rank_test_r2').sort_index(ascending=True)

In [ ]:
Y_pred.min(), Y_pred.max()

In [ ]:
Y_pred.plot()

In [ ]:
# name='ethereum'
# X = SF['ethereum'].dropna(subset=['forecast'])
# # X = SF[name].dropna(subset=['forecast']).set_index('timestamp')
# X_train, X_test, Y_train, Y_test = train_test_split(X.drop('forecast', axis=1), X['forecast'], test_size=0.3, shuffle=False)
# opti_xgbmodel = random_search.fit(X_train, Y_train, eval_set = [(X_test, Y_test)], verbose=1)
# print(f"{name} Best set of hyperparameters: ", opti_xgbmodel.best_params_)
# print(f"{name} Best score: ", opti_xgbmodel.best_score_)
# index= Y_pred.index, columns= ['prediction']), Y_test.rename("realized")], axis=1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days' )

In [ ]:
# Y_pred= pd.Series(opti_xgbmodel.predict(X_test), X_test.index).rename('prediction')
# score = root_mean_squared_error(Y_test, Y_pred)
# sm = (score)/(Y_test.mean())
# sstd= (score)/(Y_test.std())
# print(name + f' RMSE: {score} \n (score/mean):{sm} \n (score/std):{sstd}')
# pd.concat([Y_pred, Y_test.rename("realized")], axis=1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days')

In [ ]:
# xgbmodels = {}
# for name in names:
#     if name != 'bitcoin':
#         X = SF[name].dropna(subset=['forecast'])
#         X_train, X_test, Y_train, Y_test = train_test_split(X.drop('forecast', axis=1), X['forecast'], test_size=0.3, shuffle=False)
#         opti_xgbmodel = random_search.fit(X_train, Y_train, eval_set = [(X_test, Y_test)], verbose=False)
#         print(f"{name} Best set of hyperparameters: ", opti_xgbmodel.best_params_)
#         print(f"{name} Best score: ", opti_xgbmodel.best_score_)
#         Y_pred= opti_xgbmodel.predict(X_test)
#         score = root_mean_squared_error(Y_test, Y_pred)
#         sm = (score)/(Y_test.mean())
#         sstd= (score)/(Y_test.std())
#         print(name + f' RMSE: {score} \n (score/mean):{sm} \n (score/std):{sstd}')
#         pd.concat([pd.DataFrame(Y_pred, index= Y_test.index, columns= ['prediction']), Y_test.rename("realized")], axis=1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days' )
#     else: continue

scoring='neg_mean_absolute_error'
target = 'log_returns'
L1= ['BTC','ETH', 'BNB', 'MATIC', 'SOL']
timeshift = 350
days_forecast= 7
date = pd.to_datetime(dt.datetime.today(), utc =True) - pd.Timedelta(75, 'W')

binance-smart-chain Best set of hyperparameters:  {'colsample_by_tree': 0.9376835349994561, 'gamma': 0.09, 'learning_rate': 0.01, 'max_depth': 65, 'n_estimators': 50, 'num_parallel_tree': 20, 'subsample': 0.5670504225960631}
binance-smart-chain Best score:  -0.04483330485195443
days_forecast= 14

binance-smart-chain Best set of hyperparameters:  {'colsample_by_tree': 0.7476073895051453, 'gamma': 0.4, 'learning_rate': 0.05, 'max_depth': 25, 'n_estimators': 50, 'num_parallel_tree': 60, 'subsample': 0.6048059179462468}
binance-smart-chain Best score:  -0.06729950757934552

ethereum Best set of hyperparameters:  {'colsample_by_tree': 0.8167505022279493, 'gamma': 0.12, 'learning_rate': 0.01, 'max_depth': 65, 'n_estimators': 75, 'num_parallel_tree': 20, 'subsample': 0.6299390588944596}
ethereum Best score:  -0.04762516539203537

days_forecast =14
ethereum Best set of hyperparameters:  {'colsample_by_tree': 0.6644288414365456, 'gamma': 0.08, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50, 'num_parallel_tree': 60, 'subsample': 0.6945960820046824}
ethereum Best score:  -0.0631609431157224

polygon Best set of hyperparameters:  {'colsample_by_tree': 0.5633730840955109, 'gamma': 0.22, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 50, 'num_parallel_tree': 40, 'subsample': 0.5617404551326586}
polygon Best score:  -0.07776937215770342

solana Best set of hyperparameters:  {'colsample_by_tree': 0.9533658935803484, 'gamma': 0.03, 'learning_rate': 0.1, 'max_depth': 45, 'n_estimators': 50, 'num_parallel_tree': 40, 'subsample': 0.5613848962228907}
solana Best score:  -0.10050522676861658


In [ ]:
# Y_pred= random_search.predict(X_test)
# score = root_mean_squared_error(Y_test, Y_pred)
# sm = (score)/(Y_test.mean())
# sstd= (score)/(Y_test.std())
# print(name + f' RMSE: {score} \n (score/mean):{sm} \n (score/std):{sstd}')
# pd.concat([pd.DataFrame(Y_pred, index= Y_test.index, columns= ['prediction']), Y_test.rename("realized")], axis=1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days' )

In [ ]:
# Y_pred= random_search.predict(X_test)

In [ ]:
# for key in SF.keys():
#     df = SF[key].drop_duplicates()
#     X_train, X_test, Y_train, Y_test = train_test_split(df.drop('forecast', axis=1), df['forecast'], test_size=0.3, shuffle=)
#     xgbmodel = xgb.XGBRegressor(device = 'cuda').fit(X_train, Y_train, eval_set = [(X_train, Y_train), (X_test, Y_test)], verbose=False)
#     Y_pred= xgbmodel.predict(X_test)
#     score = root_mean_squared_error(Y_test, Y_pred)
#     sm = (score)/(Y_test.mean())
#     sstd= (score)/(Y_test.std())
#     print(key + f' RMSE: {score} \n (score/mean):{sm} \n (score/std):{sstd}')
#     pd.concat([pd.DataFrame(Y_pred, index= Y_test.index, columns= ['prediction']), Y_test.rename("realized")], axis=1).plot(title= f"{key}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days' )

In [ ]:
# for key in SF.keys():
#         df = SF[key].drop_duplicates()
#         x= df.drop('forecast', axis =1)
#         y = df['forecast']
#         Y_train= y.iloc[ :int(0.85*len(df))]
#         Y_test = y.iloc[int(0.85*len(df))+1: ]
#         X_train = x.iloc[ : int(0.85*len(df))]
#         X_test = x.iloc[int(0.85*len(df))+1 : ]
#         xgbmodel = xgb.XGBRegressor(device = 'cuda' ).fit(X_train, Y_train, eval_set = [(X_train, Y_train), (X_test, Y_test)], verbose=False)
#         Y_pred= xgbmodel.predict(X_test)
#         # score= np.sqrt(mean_squared_error(Y_test, Y_pred)) / Y_pred
#         score = root_mean_squared_error(Y_test, Y_pred)
#         sm = (score)/(Y_test.mean())
#         sstd= (score)/(Y_test.std())
#         print(key + f' RMSE: {score} \n (score/mean):{sm} \n (score/std):{sstd}')
#         pd.concat([pd.DataFrame(Y_pred, index= Y_test.index, columns= ['prediction']), Y_test.rename("realized")], axis=1).plot(title= f"{key}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days' )

In [ ]:
# train_xgbmodels_on_features(SF, days_forecast)